<h1>GaussianMixture</h1>

<h4>概要</h4>
混合ガウスモデル（GMM:gaussin mixture model）は、パラメータがわからない複数のガウス分布を混ぜた（足した）ものからインスタンスが生成されていることを前提とする確率的なモデルである。一つのガウス分布から生成された全てのインスタンスは、一般に楕円体のような形になるクラスタを形成する。個々のクラスタは、異なる形、サイズ、密度、向きを持つことができる

<h4>GMMの構成</h4>
GMMのクラスは複数用意されている。最も単純なGaussianMixtureクラスの場合、ガウス分布の数kをあらかじめ指定しなければならない。データセットXは、次のような確率的プロセスによって生成されたものと仮定される<br>
・個々のインスタンスに対し、k個のクラスタから無作為に一つが選択される。j番目のクラスタが選ばれる確率は、クラスタの重み$\phi^{(j)}$によって定義される。i番目のインスタンスのために選ばれたクラスタのインデックスは、$z^{(i)}$と表記する<br>
・$z^{(i)}=j$なら、つまりi番目のインスタンスがj番目のクラスタに振り分けれているなら、このインスタンスの位置${x}^{(i)}$は、平均が$\mu^{(j)}$、共分散行列が$\Sigma^{(j)}$のガウス分布から無作為にサンプリングされる。これを$x^{(j)}\sim N(\mu^{(j)}, \Sigma^{(j)})$と記述する<br>
要するに、個々の変数$z^{(i)}$は、重みが$\phi^{(j)}$のカテゴリカル分布から抽出され、個々の変数${x}^{(i)}$は、平均、共分散行列がクラスタ$z^{(i)}$によって定義される正規分布から抽出される<br>
ここで${x}^{(i)}$は、その値がわかっているため、観測変数である。対して$z^{(i)}$は、その値がわからないため、潜在変数である

<h4>推定しなければならないパラメータ</h4>
アルゴリズムが推定しなければならないパラメータは以下の通りである<br>
重み$\phi^{(j)}$、平均$\mu^{(j)}$、共分散行列$\Sigma^{(j)}$ $(j=1,\dots,k)$<br>
ここで、$\mu$と$\Sigma$はベクトルで、その内包するパラメータ数はガウス分布の次元数による

<h4>EMアルゴリズム</h4>
sklearnのGaussianMixtureモデルは、EM（期待値最大化）アルゴリズムで、パラメータを推定する。EM法は、最初にクラスタのパラメータを無作為に選び、期待値ステップ（expectation step）と最大化ステップ（maximaization step）の2ステップの処理をパラメータが収束するまで繰り返す。期待値ステップは、各インスタンスが各クラスタに属する確率を現在のクラスタパラメータに基づき推計し、最大化ステップはデータセットに含まれる全てのインスタンスを使って、当該クラスタに属する確率によりインスタンスに重みをつけて、クラスタを更新する<br>
EM法は、kmeansと同様に、非最適解に収束することがあるため、何度か実行して最良解を残す必要がある。そのためには、<b>n_init</b>に実行回数を渡せば良い。デフォルトは1のため注意

<h4>GMMのメソッド</h4>
ハードクラスタリングでは<b>predict()</b>メソッド、ソフトクラスタリングでは<b>predict_proba()</b>メソッドを使う<br>
混合ガウスモデルは生成的なモデルであるため、<b>sample()</b>メソッドを使って、モデルから新しいインスタンスをサンプリングすることができる（インスタンスはクラスタインデックス順に並べられる）<br>
<b>score_sample()</b>メソッドで任意の位置におけるモデルの密度を推計することもできる。このメソッドは、与えられた各インスタンスについて、その位置の確率密度関数の対数を推計する。そのため、スコアをexponentialの指数として使えば、そのインスタンスの位置の確率密度の値が得られる

<h4>モデルに制約を加える</h4>
次元が多く、クラスタ数が多く、インスタンスが少ない、といった場合には、EMアルゴリズムは最適解に収束するのに苦労する。そこで、アルゴリズムが学習しなければならないパラメータの数を減らす、つまりモデルに制約をかけることを考える。そのためには、<b>covariance_type</b>パラメータを次の中のどれかにする<br>
・"spherical"→全てのクラスタが円形でなければならない。ただし、直径はまちまちで良い（つまり、分散は異なっていて良い）<br>
・"diag"→クラスタは楕円形であればどのような形でもいいし、サイズもまちまちで良い。しかし、楕円体の軸は座標軸と平行でなければならない。つまり、共分散行列は対角行列でなければならない<br>
・"tied"→全てのクラスタの楕円体の形、サイズ、向きが同じでなければならない。（つまり、全てのクラスタが同じ共分散行列を共有していなければならない）<br>
<b>covariance_type</b>のデフォルトは、クラスタごとに形、サイズ、向きがばらばらで良いという意味の"full"である（クラスタごとに独自の無制約な共分散行列がある）

<h4>GMMによる異常検知</h4>
混合ガウスモデルは異常検知にも使用できる。やり方はごく簡単で、密度の低い領域にあるインスタンスを異常値と見做せば良い。ただし、混合ガウスモデルは外れ値を含む全てのデータに適合しようとするので、異常値が多すぎると、モデルの「正常性」の解釈にバイアスがかかり、外れ値の一部が正常値と見なされてしまう。そのような場合は、とりあえずモデルを訓練してみた上で、もっとも極端な外れ値を検知、除去し、少しクリーンアップされたデータセットを改めて訓練すると良い<br>
※余談だが、異常検知と新規検知は異なる。新規検知はアルゴリズムが「クリーン」なデータセットで訓練されることが前提となっている点で異常検知と異なる。異常検知にはこのような前提条件はない

<h4>クラスタ数の決め方</h4>
K平均法と同様に、慣性やシルエットスコアを使うこともできるが、クラスタが円形でなかったり、サイズがまちまちであったりすると、これらの方法は信頼性がないため、混合ガウスモデルではベイズ情報量規準（BIC）や赤池情報量規準（AIC）といった理論的な情報量規準を最小化するモデルを見つける、という方針をとる<br>
$BIC=log(m)p-2log(\hat{L})$<br>
$AIC=2p-2log(\hat{L})$<br>
mはインスタンス数、pはモデルが学習するパラメータ数、$\hat{L}$はモデルの尤度関数<br>
BICとAICは、共に学習するパラメータが多い（例えば、クラスタ数が多い）モデルにペナルティを与え、データによく適合するモデルに報酬を与える。両者に差が出る場合、BICが選ぶモデルはAICが選ぶモデルよりも単純（パラメータが少ない）なものの、データへの適合度が低いものになる傾向がある（データセットが大きいときには特にこの特徴が顕著になる）<br>
BICとAICの計算には、<b>bic()</b>、<b>aic()</b>メソッドを呼び出す

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons

In [ ]:
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=True, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

<h1>BayesianGaussianMixture</h1>